# Tutorial 5: Generate context-aware gene embedding

In [ ]:
import torch.multiprocessing as mp
import pickle
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict, OrderedDict
from tqdm import tqdm
from argparse import Namespace
import gc
import csv

from model.constants import *
import os
import torch.utils.data as data
from copy import deepcopy
import random
import helper
from sklearn.preprocessing import LabelEncoder
from omics.constants import *
import h5py
from pytorch_lightning.callbacks import (EarlyStopping, LearningRateMonitor,
                                         ModelCheckpoint, Callback)
from math import exp
import random
import h5py
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
import hashlib
import anndata as ad
from model.emb_gen.pretrain import Omics
from pytorch_lightning import LightningModule, Trainer, seed_everything
from scipy.spatial import cKDTree
from omics.baseline.imputation.preprocess.spot_input_gen import load_spot_data_with_full_info
from step1_cell_specific_gene_emb_gen import process_single_gpu_gene_pairs
from spatial_pattern_classification.spatial_pattern_cls import *
from function_prediction.run_gene_level_ours import *

## Initialize args and fix seed

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

args = Namespace(
    ckpt_path="SpotFormer_checkpoint.ckpt",
    dataset_name="cosmx_lung5_rep1",
    target_dataset="cosmx_lung5_rep1",
    gene_pct=100,
    radius=20,
    max_points=20,
    cell_gene_pairs_path="/path/to/cell_gene_pairs.csv",
    linear_hidden_dim=256,
    config="configs/bert_config.json",
    seed=42,
    f=None,
    split_slice=None,
    target_width=6724,
    target_height=5885,
    x_min=None,
    x_max=None,
    y_min=None,
    y_max=None,
    sample_ratio=1.0,
    merge_threshold=0.5,
    percentile=70
)

print('ckpt_path:', args.ckpt_path)
print(f"Dataset: {args.dataset_name}, radius: {args.radius}, max_points: {args.max_points}")

seed_everything(args.seed)

## Generate cell-specific gene embeddings

In [ ]:
cache_path = f"cached_spot_data/spot_input_spot_all_{args.dataset_name}_{args.seed}_{args.gene_pct}_{args.radius}_{args.max_points}.h5"

print("Processing cell-gene pairs for gene embeddings -> AnnData")
adata_result = process_single_gpu_gene_pairs(
    args, args.cell_gene_pairs_path, args.target_dataset, cache_path
)

## Task1: Gene spatial pattern classification

In [ ]:
adata_path = 'gene_embeddings_adata/seqfish_gene_embeddings.h5ad'
cell_gene_pairs_path = "Ground_truth/13059_2024_3217_MOESM4_ESM.csv"


results = optimized_clustering_pipeline(
    adata_path=adata_path,
    cell_gene_pairs_path=cell_gene_pairs_path,
    n_clusters_range=range(8, 25, 2),
    threshold_range=np.arange(0.3, 0.8, 0.05),
    optimize_for='combined_all',  # combine optimization
    none_method='hybrid',
    target_none_ratio=0.1735,
    save_n_clusters=14,
    output_dir='seqfish_clustering_results_with_acc'
)

print("\nPipeline completed successfully!")

if results is not None:
    print("\n" + "="*80)
    print("ACCESSING n_clusters=14 RESULTS")
    print("="*80)
    
    # Extract results for n_clusters=14
    n14_results = results['saved_n14_results']
    
    print(f"\n📦 Available data for n_clusters=14:")
    print(f"   - Cluster labels: {n14_results['cluster_labels'].shape}")
    print(f"   - Cluster centers: {n14_results['cluster_centers'].shape}")
    print(f"   - Cluster label probs: {n14_results['cluster_probs'].shape}")
    print(f"   - Cluster sizes: {n14_results['cluster_sizes'].shape}")
    print(f"   - Best threshold: {n14_results['best_threshold']:.3f}")

    cluster_labels_14 = n14_results['cluster_labels']
    cluster_centers_14 = n14_results['cluster_centers']

In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
import torch.multiprocessing as mp
import pickle
import os
from argparse import Namespace
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict, OrderedDict
from tqdm import tqdm
import gc
import csv

from omics.constants import *
import os
from omics.constants import *
import random
import h5py
from step2_unified_get_gene_emb import process_single_gpu_gene_embeddings
from step3_gene_mapping import convert_gene_names_to_ids
from step4_merge_emb import merge_embedding_csvs, merge_gene_lists

## Generate universal gene embeddings

### Get gene list for function prediction benchmark

#### Peek gene list from benchmarks

In [ ]:
import pickle
import pandas as pd
import numpy as np
import os

def extract_genes_from_benchmark_results(result_files):
    """extract all used genes from benchmark results"""
    all_genes = set()
    
    for file_path in result_files:
        print(f"Processing file: {file_path}")
        
        try:
            with open(file_path, 'rb') as f:
                results = pickle.load(f)
            
            for disease_term, result_df in results.items():
                print(f"  - {disease_term}: {len(result_df)} embedding methods")
                
                
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    return all_genes

# List of benchmark result files
result_files = [
    "../gene-embedding-benchmarks/results/gene_level/go_all_holdout_results.pkl",
    "../gene-embedding-benchmarks/results/gene_level/go_all_holdout_results_after_2020.pkl", 
    "../gene-embedding-benchmarks/results/gene_level/omim_holdout_results.pkl"
]

# Inspect pickle file structure
def inspect_pickle_structure(file_path):
    """Check the structure of pickle file"""
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    
    print(f"\n=== {file_path} Structure Analysis ===")
    print(f"Data type: {type(data)}")
    
    if isinstance(data, dict):
        print(f"Number of dictionary keys: {len(data)}")
        print("First few keys:", list(data.keys())[:3])
        
        # Check the structure of the first value
        first_key = list(data.keys())[0]
        first_value = data[first_key]
        print(f"\nFirst value ({first_key}) type: {type(first_value)}")
        
        if isinstance(first_value, pd.DataFrame):
            print(f"DataFrame shape: {first_value.shape}")
            print(f"Column names: {first_value.columns.tolist()}")
            print(f"First few indices: {first_value.index[:3].tolist()}")
            print(f"Example data:\n{first_value.head()}")
            
    return data

# Check the structure of each file
for file_path in result_files:
    if os.path.exists(file_path):
        try:
            data = inspect_pickle_structure(file_path)
        except Exception as e:
            print(f"Cannot read {file_path}: {e}")
    else:
        print(f"File not found: {file_path}")

#### Get gene list from benchmarks

In [ ]:
import pickle
import pandas as pd
import os
import warnings

def read_pickle_with_compatibility(file_path):
    
    # Method 1: Standard pickle reading
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        return data
    except Exception as e1:
        print(f"    Error: Standard pickle reading failed: {str(e1)[:100]}...")
        
        # Method 2: Ignore pandas version warnings
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                with open(file_path, 'rb') as f:
                    data = pickle.load(f)
                return data
        except Exception as e2:
            print(f"    Error: Ignoring warnings failed: {str(e2)[:100]}...")
            
            # Method 3: Use pandas to read directly
            try:
                data = pd.read_pickle(file_path)
                return data
            except Exception as e3:
                print(f"    Error: pandas reading failed: {str(e3)[:100]}...")
                
                # Method 4: Degrade processing - try to reconstruct DataFrame
                try:
                    with open(file_path, 'rb') as f:
                        # Use pickle底层协议
                        import pickle
                        data = pickle.load(f)
                        
                        # If it's a dictionary, try to fix the DataFrame
                        if isinstance(data, dict):
                            fixed_data = {}
                            for key, value in data.items():
                                if hasattr(value, 'values') and hasattr(value, 'columns'):
                                    # Try to reconstruct DataFrame
                                    try:
                                        if hasattr(value, 'index'):
                                            new_df = pd.DataFrame(
                                                data=value.values,
                                                columns=value.columns,
                                                index=value.index
                                            )
                                        else:
                                            new_df = pd.DataFrame(
                                                data=value.values,
                                                columns=value.columns
                                            )
                                        fixed_data[key] = new_df
                                    except:
                                        # If reconstruction fails, skip this entry
                                        continue
                                else:
                                    fixed_data[key] = value
                            return fixed_data
                        else:
                            return data
                            
                except Exception as e4:
                    print(f"    Error: Reconstructing reading failed: {str(e4)[:100]}...")
                    return None

def extract_genes_from_dataframe(df):
    """Extract genes from DataFrame safely"""
    try:
        if isinstance(df, pd.DataFrame) and 'gene' in df.columns:
            # Ensure gene column is a string type
            genes = df['gene'].astype(str).tolist()
            # Filter out empty values and NaN
            genes = [g for g in genes if g and g != 'nan' and g != 'None']
            return set(genes)
        else:
            return set()
    except Exception as e:
        print(f"    Error: Extracting genes: {e}")
        return set()

def extract_genes_from_fold_data():
    """Extract gene list from fold data"""
    
    # All possible file locations
    possible_files = [
        "../gene-embedding-benchmarks/bin/gene_level/GO/go_cv_fold1_dict.pkl", 
        "../gene-embedding-benchmarks/bin/gene_level/GO/go_cv_fold2_dict.pkl", 
        "../gene-embedding-benchmarks/bin/gene_level/GO/go_cv_fold3_dict.pkl", 
        "../gene-embedding-benchmarks/bin/gene_level/GO/go_holdout_dict.pkl", 
        "../gene-embedding-benchmarks/bin/gene_level/GO_after_2020/go_cv_fold1_dict_after_2020.pkl", 
        "../gene-embedding-benchmarks/bin/gene_level/GO_after_2020/go_cv_fold2_dict_after_2020.pkl", 
        "../gene-embedding-benchmarks/bin/gene_level/GO_after_2020/go_cv_fold3_dict_after_2020.pkl", 
        "../gene-embedding-benchmarks/bin/gene_level/GO_after_2020/go_holdout_dict_after_2020.pkl",
        "../gene-embedding-benchmarks/bin/gene_level/OMIM/omim_cv_fold1_dict.pkl",
        "../gene-embedding-benchmarks/bin/gene_level/OMIM/omim_cv_fold2_dict.pkl",
        "../gene-embedding-benchmarks/bin/gene_level/OMIM/omim_cv_fold3_dict.pkl",
        "../gene-embedding-benchmarks/bin/gene_level/OMIM/omim_holdout_dict.pkl"
    ]
    
    all_genes = set()
    successful_files = []
    failed_files = []
    
    for file_path in possible_files:
        if os.path.exists(file_path):
            print(f"\n=== {file_path} ===")
            
            # Use compatibility reading
            data = read_pickle_with_compatibility(file_path)
            
            if data is not None:
                print(f"Data type: {type(data)}")
                
                if isinstance(data, dict):
                    print(f"Number of diseases/processes: {len(data)}")
                    
                    # Check the structure of the first entry
                    if len(data) > 0:
                        first_key = list(data.keys())[0]
                        first_value = data[first_key]
                        print(f"First entry ({first_key}) type: {type(first_value)}")
                        
                        if isinstance(first_value, pd.DataFrame):
                            print(f"DataFrame columns: {first_value.columns.tolist()}")
                            print(f"DataFrame shape: {first_value.shape}")
                            if 'gene' in first_value.columns:
                                print(f"Contains gene column! First few genes: {first_value['gene'].head().tolist()}")
                    
                    # Extract all genes
                    file_genes = set()
                    successful_terms = 0
                    
                    for term, df in data.items():
                        genes_in_term = extract_genes_from_dataframe(df)
                        if genes_in_term:
                            file_genes.update(genes_in_term)
                            successful_terms += 1
                    
                    if file_genes:
                        print(f"Extracted {len(file_genes)} genes from this file (from {successful_terms} entries)")
                        all_genes.update(file_genes)
                        successful_files.append(file_path)
                    else:
                        print("No gene information found in this file")
                        failed_files.append((file_path, "No gene information"))
                else:
                    print(f"Data is not a dictionary format: {type(data)}")
                    failed_files.append((file_path, "Data format error"))
            else:
                print("All reading methods failed")
                failed_files.append((file_path, "Reading failed"))
                
        else:
            print(f"File not found: {file_path}")
            failed_files.append((file_path, "File not found"))
    
    return all_genes, successful_files, failed_files

# Extract genes from fold data
print("Extracting genes from fold data...")
benchmark_genes, successful_files, failed_files = extract_genes_from_fold_data()

if benchmark_genes:
    print(f"\n=== Summary of extraction results ===")
    print(f"Total unique genes found: {len(benchmark_genes)}")
    print(f"Successful processed files: {len(successful_files)}")
    print(f"Failed files: {len(failed_files)}")
    
    # Show some example genes
    sample_genes = sorted(list(benchmark_genes))[:10]
    print(f"Example genes: {sample_genes}")
    
    # Save gene list
    with open('benchmark_gene_list_complete.txt', 'w') as f:
        for gene in sorted(benchmark_genes):
            f.write(f"{gene}\n")
    
    print("\nGene list saved to benchmark_gene_list_complete.txt")
    
    # Count by file type
    print(f"\n=== Successfully extracted sources ===")
    go_files = 0
    omim_files = 0
    for file_path in successful_files:
        filename = os.path.basename(file_path)
        print(f"  ✓ {filename}")
        if 'omim' in filename.lower():
            omim_files += 1
        else:
            go_files += 1
    
    print(f"\nGO files: {go_files}, OMIM files: {omim_files}")
    
    # Show failed files
    if failed_files:
        print(f"\n=== Failed files ===")
        for file_path, reason in failed_files:
            filename = os.path.basename(file_path)
            print(f"  ✗ {filename}: {reason}")
        
else:
    print("Cannot extract gene information, please check other file locations")
    
    # Show all failed reasons
    print("\nFailed details:")
    for file_path, reason in failed_files:
        print(f"  {os.path.basename(file_path)}: {reason}")

#### Convert gene ID into gene name

In [ ]:
import mygene
import pandas as pd

def convert_entrez_to_symbols(entrez_file):
    """Convert Entrez ID to gene symbols"""
    
    # Read gene list
    with open(entrez_file, 'r') as f:
        entrez_ids = [line.strip() for line in f if line.strip()]
    
    print(f"Preparing to convert {len(entrez_ids)} Entrez IDs")
    
    # Initialize mygene
    mg = mygene.MyGeneInfo()
    
    # Batch conversion (avoid API limit)
    batch_size = 1000
    gene_mapping = {}
    failed_ids = []
    
    for i in range(0, len(entrez_ids), batch_size):
        batch = entrez_ids[i:i+batch_size]
        batch_num = i//batch_size + 1
        total_batches = (len(entrez_ids)-1)//batch_size + 1
        
        print(f"Processing batch {batch_num}/{total_batches} ({len(batch)} genes)")
        
        try:
            results = mg.querymany(
                batch, 
                scopes='entrezgene', 
                fields='symbol,entrezgene,name', 
                species='human',
                returnall=True
            )
            
            # Process successful results
            for result in results['out']:
                if 'symbol' in result and 'entrezgene' in result:
                    entrez_id = str(result['entrezgene'])
                    symbol = result['symbol']
                    gene_name = result.get('name', '')
                    gene_mapping[entrez_id] = {
                        'symbol': symbol,
                        'name': gene_name
                    }
                elif 'query' in result:
                    # Record failed IDs
                    failed_ids.append(result['query'])
                    
        except Exception as e:
            print(f"Batch {batch_num} conversion failed: {e}")
            failed_ids.extend(batch)
    
    print(f"\nConversion completed!")
    print(f"Successfully converted: {len(gene_mapping)} genes")
    print(f"Conversion failed: {len(failed_ids)} genes")
    print(f"Conversion success rate: {len(gene_mapping)/len(entrez_ids)*100:.1f}%")
    
    return gene_mapping, failed_ids

print("Starting gene ID conversion...")
gene_mapping, failed_ids = convert_entrez_to_symbols('benchmark_gene_list_complete.txt')

if gene_mapping:
    mapping_df = pd.DataFrame([
        {
            'entrez_id': entrez_id,
            'gene_symbol': info['symbol'], 
            'gene_name': info['name']
        }
        for entrez_id, info in gene_mapping.items()
    ])
    
    mapping_df.to_csv('benchmark_gene_mapping.csv', index=False)
    print(f"Detailed mapping table saved to benchmark_gene_mapping.csv")
    
    symbols = sorted([info['symbol'] for info in gene_mapping.values()])
    with open('benchmark_gene_symbols.txt', 'w') as f:
        for symbol in symbols:
            f.write(f"{symbol}\n")
    
    print(f"\nExample conversion results:")
    for i, (entrez_id, info) in enumerate(list(gene_mapping.items())[:10]):
        print(f"  {entrez_id} -> {info['symbol']} ({info['name'][:50]}...)")
    
    if failed_ids:
        with open('failed_entrez_ids.txt', 'w') as f:
            for failed_id in failed_ids:
                f.write(f"{failed_id}\n")
        print(f"\n{len(failed_ids)} failed IDs saved to failed_entrez_ids.txt")


#### map into our dataset's gene list

In [ ]:
merfish_adata = sc.read_h5ad('merfish_processed.h5ad')
merfish_adata

In [ ]:
cosmx_data = pd.read_pickle('CosMx/spot_dataframe.pkl')
CosMx_gene = np.unique(cosmx_data['gene'])

In [ ]:
def extract_and_compare_genes():
    """Extract genes from actual data and compare with benchmark"""
    
    # Read benchmark test genes
    with open('benchmark_gene_symbols.txt', 'r') as f:
        benchmark_genes = set(line.strip() for line in f if line.strip())
    
    print(f"Total benchmark test genes: {len(benchmark_genes)}")
    
    # === Method 1: If you have merfish_adata object ===
    try:
        # Assuming your merfish_adata is already loaded
        merfish_genes = list(merfish_adata.var_names)
        # Filter out non-target genes
        merfish_genes = [g for g in merfish_genes if not g.startswith('notarget')]
        print(f"MERFISH genes (filtered): {len(merfish_genes)}")
    except:
        print("Please load merfish_adata first")
        merfish_genes = []
    
    # === Method 2: If you have CosMx data file ===
    try:
        # Read from pickle file
        cosmx_data = pd.read_pickle('CosMx/spot_dataframe.pkl')
        cosmx_genes = list(np.unique(cosmx_data['gene']))
        print(f"CosMx genes: {len(cosmx_genes)}")
    except:
        print("Cannot read CosMx data file")
        cosmx_genes = []
    
    # Perform matching analysis
    if merfish_genes or cosmx_genes:
        # Convert to uppercase for matching
        benchmark_upper = set(g.upper() for g in benchmark_genes)
        
        results = {}
        
        if merfish_genes:
            merfish_upper = set(g.upper() for g in merfish_genes)
            merfish_overlap = merfish_upper.intersection(benchmark_upper)
            
            print(f"\n=== MERFISH matching results ===")
            print(f"MERFISH genes: {len(merfish_genes)}")
            print(f"Overlap with benchmark: {len(merfish_overlap)}")
            print(f"Overlap rate: {len(merfish_overlap)/len(benchmark_upper)*100:.1f}%")
            
            sample_overlap = sorted(list(merfish_overlap))[:10]
            print(f"Overlapping genes example: {sample_overlap}")
            
            results['merfish'] = {
                'total': len(merfish_genes),
                'overlap': len(merfish_overlap),
                'overlap_genes': sorted(merfish_overlap)
            }
        
        if cosmx_genes:
            cosmx_upper = set(g.upper() for g in cosmx_genes)
            cosmx_overlap = cosmx_upper.intersection(benchmark_upper)
            
            # Show some overlapping genes
            sample_overlap = sorted(list(cosmx_overlap))[:10]
            print(f"Overlapping genes example: {sample_overlap}")
            
            results['cosmx'] = {
                'total': len(cosmx_genes),
                'overlap': len(cosmx_overlap),
                'overlap_genes': sorted(cosmx_overlap)
            }
        
        # If both datasets exist, compare them
        if merfish_genes and cosmx_genes:
            merfish_upper = set(g.upper() for g in merfish_genes)
            cosmx_upper = set(g.upper() for g in cosmx_genes)
            
            both_overlap = merfish_upper.intersection(cosmx_upper)
            three_way_overlap = merfish_overlap.intersection(cosmx_overlap)
            
            print(f"\n=== Comparison between datasets ===")
            print(f"MERFISH and CosMx overlap: {len(both_overlap)}")
            print(f"Three-way overlap (all match benchmark): {len(three_way_overlap)}")
            
            results['comparison'] = {
                'merfish_cosmx_overlap': len(both_overlap),
                'three_way_overlap': len(three_way_overlap),
                'three_way_genes': sorted(three_way_overlap)
            }
        
        return results
    
    else:
        print("No available gene data for comparison")
        return None

results = extract_and_compare_genes()

#### Save matched genes

In [ ]:
def save_matched_genes_for_embedding():
    
    with open('benchmark_gene_symbols.txt', 'r') as f:
        benchmark_genes = set(line.strip() for line in f if line.strip())
    
    try:
        # MERFISH
        merfish_genes = list(merfish_adata.var_names)
        merfish_genes_filtered = [g for g in merfish_genes if not g.startswith('notarget')]
        
        # CosMx
        cosmx_data = pd.read_pickle('CosMx/spot_dataframe.pkl')
        cosmx_genes = list(np.unique(cosmx_data['gene']))
        
        # Calculate overlap
        benchmark_upper = set(g.upper() for g in benchmark_genes)
        merfish_upper = set(g.upper() for g in merfish_genes_filtered)
        cosmx_upper = set(g.upper() for g in cosmx_genes)
        
        merfish_overlap = merfish_upper.intersection(benchmark_upper)
        cosmx_overlap = cosmx_upper.intersection(benchmark_upper)
        
        # Save overlapping genes
        with open('merfish_benchmark_genes.txt', 'w') as f:
            for gene in sorted(merfish_overlap):
                f.write(f"{gene}\n")
        
        with open('cosmx_benchmark_genes.txt', 'w') as f:
            for gene in sorted(cosmx_overlap):
                f.write(f"{gene}\n")
        
        # Save combined gene list (for complete benchmark)
        combined_genes = merfish_overlap.union(cosmx_overlap)
        with open('combined_benchmark_genes.txt', 'w') as f:
            for gene in sorted(combined_genes):
                f.write(f"{gene}\n")
        
        # Find three-way overlap (for direct comparison)
        three_way_overlap = merfish_overlap.intersection(cosmx_overlap)
        with open('three_way_overlap_genes.txt', 'w') as f:
            for gene in sorted(three_way_overlap):
                f.write(f"{gene}\n")

        return {
            'merfish_genes': sorted(merfish_overlap),
            'cosmx_genes': sorted(cosmx_overlap),
            'combined_genes': sorted(combined_genes),
            'three_way_genes': sorted(three_way_overlap)
        }
        
    except Exception as e:
        print(f"Error processing gene data: {e}")
        return None

gene_lists = save_matched_genes_for_embedding()

#### filter new gene list

In [ ]:
def prepare_test_subsets_with_compatibility():
    
    import pickle
    import os
    import pandas as pd
    
    with open('combined_benchmark_genes.txt', 'r') as f:
        target_genes = set(line.strip().upper() for line in f)
    
    print(f"Target gene number: {len(target_genes)}")
    
    gene_mapping = pd.read_csv('benchmark_gene_mapping.csv')
    entrez_to_symbol = dict(zip(gene_mapping['entrez_id'].astype(str), 
                               gene_mapping['gene_symbol'].str.upper()))
    
    benchmark_files = [
        "../gene-embedding-benchmarks/bin/gene_level/GO/go_cv_fold1_dict.pkl",
        "../gene-embedding-benchmarks/bin/gene_level/GO/go_cv_fold2_dict.pkl", 
        "../gene-embedding-benchmarks/bin/gene_level/GO/go_cv_fold3_dict.pkl",
        "../gene-embedding-benchmarks/bin/gene_level/GO/go_holdout_dict.pkl",
        "../gene-embedding-benchmarks/bin/gene_level/OMIM/omim_cv_fold1_dict.pkl",
        "../gene-embedding-benchmarks/bin/gene_level/OMIM/omim_cv_fold2_dict.pkl",
        "../gene-embedding-benchmarks/bin/gene_level/OMIM/omim_cv_fold3_dict.pkl", 
        "../gene-embedding-benchmarks/bin/gene_level/OMIM/omim_holdout_dict.pkl"
    ]
    
    os.makedirs('test_subsets', exist_ok=True)
    
    subset_stats = []
    
    for file_path in benchmark_files:
        if os.path.exists(file_path):
            filename = os.path.basename(file_path)
            print(f"\nProcessing {filename}...")
            
            try:
                with open(file_path, 'rb') as f:
                    original_data = pickle.load(f)
                
            except Exception as e1:
                print(f"Error: {e1}")
                
                try:
                    with open(file_path, 'rb') as f:
                        original_data = pickle.load(f)
                        
                except Exception as e2:
                    print(f"Protocol=2 read failed: {e2}")
                    
                    try:
                        original_data = pd.read_pickle(file_path)
                        
                    except Exception as e3:
                        print(f"pd.read_pickle read failed: {e3}")
                        continue
            
            # If successfully read, process
            try:
                subset_data = {}
                total_samples = 0
                kept_samples = 0
                
                for term, data in original_data.items():
                    # Process different data formats
                    if isinstance(data, pd.DataFrame):
                        df = data
                    elif hasattr(data, 'to_frame'):
                        df = data.to_frame()
                    elif isinstance(data, dict):
                        df = pd.DataFrame(data)
                    else:
                        # Try to convert to DataFrame
                        try:
                            df = pd.DataFrame(data)
                        except:
                            print(f"Cannot process term {term} data format")
                            continue
                    
                    if 'gene' in df.columns:
                        total_samples += len(df)
                        
                        # Create new DataFrame to avoid version issues
                        new_df = pd.DataFrame({
                            'gene': df['gene'].values,
                            'result': df['result'].values if 'result' in df.columns else [1] * len(df)
                        })
                        
                        # Convert gene ID
                        new_df['gene_symbol'] = new_df['gene'].astype(str).map(entrez_to_symbol)
                        
                        # Filter target genes
                        mask = new_df['gene_symbol'].isin(target_genes)
                        subset_df = new_df[mask][['gene', 'result']].copy()
                        
                        if len(subset_df) > 0:
                            subset_data[term] = subset_df
                            kept_samples += len(subset_df)
                
                # Save subset file
                if subset_data:
                    output_filename = filename.replace('.pkl', '_subset.pkl')
                    output_path = os.path.join('test_subsets', output_filename)
                    
                    with open(output_path, 'wb') as f:
                        pickle.dump(subset_data, f, protocol=2)  # Use better compatible protocol
                    
                    subset_stats.append({
                        'file': output_filename,
                        'terms': len(subset_data),
                        'original_samples': total_samples,
                        'kept_samples': kept_samples,
                        'retention_rate': f"{kept_samples/total_samples*100:.1f}%" if total_samples > 0 else "0%"
                    })
                    
                    print(f"{output_filename}: {len(subset_data)} terms, {kept_samples}/{total_samples} samples")
                else:
                    print(f"{filename}: no matching data")

            except Exception as e:
                print(f"{filename}: error processing data: {e}")
    
    print(f"\n=== Summary of subset creation ===")
    print(f"Successfully created {len(subset_stats)} test subset files")
    
    for stat in subset_stats:
        print(f"  {stat['file']}: {stat['terms']} terms, {stat['retention_rate']} retention")
    
    return subset_stats

subset_stats = prepare_test_subsets_with_compatibility()

### Generate gene embeddings for each dataset

In [ ]:
args = Namespace(
    ckpt_path="SpotFormer_checkpoint.ckpt",
    dataset_name="cosmx_lung5_rep1",
    target_dataset="cosmx_lung5_rep1",
    gene_pct=100,
    linear_hidden_dim='256',
    radius=20,
    max_points=20,
    config='configs/bert_config.json',
    seed=42,
    f=None,
    split_slice=None,
    target_width=6724,
    target_height=5885,
    x_min=None,
    x_max=None,
    y_min=None,
    y_max=None,
    sample_ratio=1.0,
    merge_threshold=0.5,
    percentile=70,
    gene_list_path="cosmx_benchmark_genes.txt"
)

args.linear_hidden_dim = '256'
args.radius = 20
args.max_points = 20
args.config = 'configs/bert_config.json'
print(f"Dataset: {args.dataset_name}, radius: {args.radius}, max_points: {args.max_points}")

if args.dataset_name == 'cosmx_lung5_rep1':
    cache_path = f'spot_input_spot_sampled_{args.dataset_name}_{args.seed}_{args.gene_pct}_{args.radius}_{args.max_points}_0.1.h5'
else:
    cache_path = f'spot_input_spot_all_{args.dataset_name}_{args.seed}_{args.gene_pct}_{args.radius}_{args.max_points}.h5'

print("Processing gene-level embeddings from gene list -> CSV format")
emb_file, genelist_file = process_single_gpu_gene_embeddings(
    args, args.gene_list_path, args.target_dataset, cache_path
)

# Construct cache path
if args.dataset_name == 'cosmx_lung5_rep1':
    cache_path = f'spot_input_spot_sampled_{args.dataset_name}_{args.seed}_{args.gene_pct}_{args.radius}_{args.max_points}_0.1.h5'
else:
    cache_path = f'spot_input_spot_all_{args.dataset_name}_{args.seed}_{args.gene_pct}_{args.radius}_{args.max_points}.h5'

print("Processing gene-level embeddings from gene list -> CSV format")
emb_file, genelist_file = process_single_gpu_gene_embeddings(
    args, args.gene_list_path, args.target_dataset, cache_path
)

### Convert gene name list to gene ID list using benchmark gene mapping

In [ ]:
args = Namespace(
    genelist_file='merfish_gene100pct_Spotformer_genelist.txt',
    mapping_file="benchmark_gene_mapping.csv",
    output_file='Spotformer_genelist2.txt'
)

if args.output_file is None:
    base_name = os.path.splitext(args.genelist_file)[0]
    if base_name.endswith('_genelist'):
        base_name = base_name[:-9]
    args.output_file = f"{base_name}_geneid.txt"

print(f"Converting gene names to IDs...")
print(f"  Input: {args.genelist_file}")
print(f"  Mapping: {args.mapping_file}")
print(f"  Output: {args.output_file}")

gene_ids, matched_count, unmatched_genes = convert_gene_names_to_ids(
    args.genelist_file, args.mapping_file, args.output_file
)

print(f"\nDone! Generated {len(gene_ids)} gene IDs with {matched_count} matches.")


### Merge gene embeddings from different datasets

In [ ]:
args = Namespace(
    emb_file1="cosmx_lung5_rep1_gene20pct_Spotformer_emb.csv",
    emb_file2="merfish_gene100pct_Spotformer_emb.csv",
    gene_file1="Spotformer_genelist.txt",
    gene_file2="Spotformer_genelist2.txt",
    output_dir="gene_embeddings_csv/",
    output_prefix="merged"
)

merged_emb_file = os.path.join(args.output_dir, f"{args.output_prefix}_Spotformer_emb.csv")
merged_gene_file = os.path.join(args.output_dir, f"{args.output_prefix}_Spotformer_genelist.txt")

print("=" * 60)
print("MERGING EMBEDDING FILES")
print("=" * 60)

# Merge embedding files
merged_embeddings = merge_embedding_csvs(
    args.emb_file1, args.emb_file2, merged_emb_file
)

print("\n" + "=" * 60)
print("MERGING GENE LIST FILES") 
print("=" * 60)

# Merge gene list files
merged_genes = merge_gene_lists(
    args.gene_file1, args.gene_file2, merged_gene_file
)

print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)

# Verify consistency
if len(merged_genes) == len(merged_embeddings):
    print(f"✓ Consistency check passed: {len(merged_genes)} genes match {len(merged_embeddings)} embeddings")
else:
    print(f"✗ Consistency check failed: {len(merged_genes)} genes vs {len(merged_embeddings)} embeddings")

print(f"\nOutput files:")
print(f"  Merged embeddings: {merged_emb_file}")
print(f"  Merged gene list: {merged_gene_file}")


## Task2: Gene function prediction

In [ ]:
try:
    # 1. Extract embeddings
    (
        reindexed_embeddings,
        gene_lists,
        reference_node2index,
        reference_genes,
    ) = helper.get_embeddings('embeddings/intersect/ours')
    
    print(f"Successfully loaded embeddings:")
    for subfolder, embedding_df in reindexed_embeddings.items():
        print(f"  {os.path.basename(subfolder)}: shape {embedding_df.shape}")
    print(f"Number of genes: {len(reference_genes)}")
    
except Exception as e:
    print(f"Failed to load embeddings: {e}")
    exit(1)

try:
    # 2. Load test subsets
    go_data = load_test_subsets()
    
except Exception as e:
    print(f"Failed to load test subsets: {e}")
    exit(1)

# 3. GO benchmark
if go_data:
    try:
        print("\n=== Running GO safe benchmark ===")
        
        go_combined = combine_all_folds(
            go_data['cv_fold1'],
            go_data['cv_fold2'],
            go_data['cv_fold3'],
            go_data['holdout']
        )
        
        print(f"GO combined data: {len(go_combined)} terms")
        
        go_results, go_stats = run_safe_benchmark(reindexed_embeddings, go_combined)
        
        # Safe results
        os.makedirs("results_subset", exist_ok=True)
        with open("results_subset/go_safe_results.pkl", "wb") as f:
            pickle.dump(go_results, f)
        
        with open("results_subset/go_stats.pkl", "wb") as f:
            pickle.dump(go_stats, f)
        
        # Summarize results
        go_performance = summarize_results(go_results, "GO")
        
    except Exception as e:
        print(f"Failed to run GO benchmark: {e}")
        import traceback
        traceback.print_exc()
